In [2]:
!pip install pyspark

     |████████████████████████████████| 204.7MB 66kB/s 
     |████████████████████████████████| 204kB 19.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=2dc319bad8ba62c99f61e7488d00a41a8c4ceae159067e2c865b4429035559a0
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [3]:
#Initialize SparkSession and SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [4]:
#Create a Spark Session
SpSession = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
#Get the Spark Context from Spark Session    
SpContext = SpSession.sparkContext

In [20]:
#Load the data file in ALS format (user, item, rating)
ratingsData = SpContext.textFile("/content/drive/My Drive/UserItemData.txt")
ratingsData.collect()


['1001,9001,10',
 '1001,9002,1',
 '1001,9003,9',
 '1002,9001,3',
 '1002,9002,5',
 '1002,9003,1',
 '1002,9004,10',
 '1003,9001,2',
 '1003,9002,6',
 '1003,9003,2',
 '1003,9004,9',
 '1003,9005,10',
 '1003,9006,8',
 '1003,9007,9',
 '1004,9001,9',
 '1004,9002,2',
 '1004,9003,8',
 '1004,9004,3',
 '1004,9010,10',
 '1004,9011,9',
 '1004,9012,8',
 '1005,9001,8',
 '1005,9002,3',
 '1005,9003,7',
 '1005,9004,1',
 '1005,9010,9',
 '1005,9011,10',
 '1005,9012,9',
 '1005,9013,8',
 '1005,9014,1',
 '1005,9015,1',
 '1006,9001,7',
 '1006,9002,4',
 '1006,9003,8',
 '1006,9004,1',
 '1006,9010,7',
 '1006,9011,6',
 '1006,9012,9']

In [21]:
#Convert the strings into a proper vector
ratingVector=ratingsData.map(lambda l: l.split(','))\
        .map(lambda l:(int(l[0]), int(l[1]), float(l[2])))

#Build a SQL Dataframe
ratingsDf=SpSession.createDataFrame(ratingVector, \
            ["user","item","rating"])


In [22]:
#build the model based on ALS
from pyspark.ml.recommendation import ALS
als = ALS(rank=10, maxIter=5)
model = als.fit(ratingsDf)

In [23]:

model.userFactors.orderBy("id").collect()

[Row(id=1001, features=[-0.7111058831214905, -0.4655403792858124, -0.5526246428489685, 0.05449123680591583, 0.10721896588802338, 0.15857446193695068, -0.9199193120002747, 0.9160863757133484, 0.827067494392395, 0.22367244958877563]),
 Row(id=1002, features=[-0.053237274289131165, 1.1351085901260376, 0.2518637776374817, -0.7319645285606384, 1.1256179809570312, -0.21404491364955902, -0.000969590968452394, 0.5095397233963013, -0.32465288043022156, 0.314593106508255]),
 Row(id=1003, features=[0.10029527544975281, 1.0456218719482422, 0.2734657824039459, -0.04982317239046097, 0.8296935558319092, -0.5945821404457092, -0.2038547396659851, 0.42615535855293274, -0.6098006963729858, 0.10262810438871384]),
 Row(id=1004, features=[-0.42074036598205566, -0.03923485428094864, -0.36363741755485535, -0.38663432002067566, 0.3423798084259033, -0.03477495536208153, -0.9704579710960388, 0.893308162689209, 0.6319288611412048, 0.1686072051525116]),
 Row(id=1005, features=[-0.9128438830375671, 0.01002091169357

In [24]:
#Create a test data set of users and items you want ratings for
testDf = SpSession.createDataFrame(   \
        [(1001, 9003),(1001,9004),(1001,9005)], \
        ["user","item"])

#Predict            
predictions=(model.transform(testDf).collect())
predictions

[Row(user=1001, item=9004, prediction=-0.46119219064712524),
 Row(user=1001, item=9005, prediction=-2.4024291038513184),
 Row(user=1001, item=9003, prediction=8.996182441711426)]